# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [ ]:
from google.colab import userdata
import os
from openai import OpenAI

# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# client = OpenAI(api_key=OPENAI_API_KEY)

# 환경변수 지정
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

In [ ]:
def chat():
  for i in range(10):
    user_input = input("챗팅을 입력해보세요(종료를 원하시면 종료를 입력해주세요.):")
    if user_input == '종료':
      print("채팅이 종료되었습니다.")
      break
    else:
      chat_history, response = chat_completion(user_input)
      print(response)

chat()

챗팅을 입력해보세요(종료를 원하시면 종료를 입력해주세요.):언녕
안녕하세요! 어떻게 도와드릴까요?
챗팅을 입력해보세요(종료를 원하시면 종료를 입력해주세요.):종료
채팅이 종료되었습니다.


In [ ]:
response

'다양한 질문에 답변하고, 정보 제공, 조언, 학습 도움, 그리고 심지어 간단한 대화도 할 수 있어요. 궁금한 점이나 필요하신 것이 있다면 말씀해 주세요!'

In [ ]:
#

### 대화형 챗

In [ ]:
def chat_completion(message, chat_history=None):
    # chat_history가 처음이면 초기화 + system message 포함
    if chat_history is None:
        chat_history = [
            {'role': 'system', 'content': '너는 친절한 챗봇이야.'}
        ]

    # 새 user 메시지 추가
    chat_history.append({'role': 'user', 'content': message})

    # GPT 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=chat_history
    )

    # assistant 응답 추출 + 저장
    reply = response.choices[0].message.content
    chat_history.append({'role': 'assistant', 'content': reply})

    return chat_history, reply

In [ ]:
print(chat_history, reply)

[{'role': 'system', 'content': '너는 친절한 챗봇이야.'}, {'role': 'user', 'content': '안녕'}, {'role': 'assistant', 'content': '안녕하세요! 어떻게 도와드릴까요?'}] 안녕하세요! 어떻게 도와드릴까요?


In [ ]:
chat_history, response = chat_completion('뭘 도와줄 수 있는데?', chat_history)

print(response)

저는 여러 가지를 도와드릴 수 있어요! 예를 들어:

- **질문에 답하기**: 다양한 주제에 대한 질문에 대해 정보를 제공할 수 있습니다.
- **학습 도움**: 공부하고 있는 내용에 대해 설명해드리거나 추가 자료를 추천할 수 있어요.
- **생활 팁 제공**: 요리, 여행, 건강 등 일상 생활에서 유용한 팁이나 조언을 드릴 수 있습니다.
- **문제 해결**: 어떤 문제에 대해 함께 고민하고 해결책을 제안해 드릴 수 있어요.
- **일상 대화**: 이야기를 나누고 싶을 때, 편안하게 대화할 수 있습니다.

무엇을 도와드릴까요? 궁금한 점이 있다면 말씀해 주세요!


In [ ]:
chat_history

[{'role': 'system', 'content': '너는 친절한 챗봇이야.'},
 {'role': 'user', 'content': '뭘 도와줄 수 있는데?'},
 {'role': 'assistant',
  'content': '저는 여러 가지 질문에 답하고 정보를 제공하는 데 도움을 줄 수 있어요! 일반적인 지식, 학습, 여행 정보, 건강, 기술, 문화, 요리 레시피 등 다양한 주제에 대한 상담이 가능합니다. 궁금한 점이나 필요한 정보가 있으면 말씀해 주세요!'},
 {'role': 'user', 'content': '뭘 도와줄 수 있는데?'},
 {'role': 'assistant',
  'content': '저는 다양한 주제에 대해 도움을 드릴 수 있습니다. 예를 들어:\n\n1. **정보 제공**: 역사, 과학, 기술, 문화 등 다양한 분야에 대한 정보를 제공할 수 있어요.\n2. **학습 도움**: 특정 주제에 대해 설명해주거나, 공부하는 데 필요한 자료를 찾는 데 도와줄 수 있습니다.\n3. **조언 및 팁**: 여행, 요리, 건강 및 라이프스타일과 관련된 조언을 줄 수 있어요.\n4. **문제 해결**: 특정 문제에 대한 해결 방법을 제안해드릴 수 있습니다.\n5. **일상 질문**: 일반적인 질문이나 호기심을 해결하는 데 도움을 줄 수 있습니다.\n\n궁금한 점이 있다면 말씀해 주세요!'},
 {'role': 'user', 'content': '뭘 도와줄 수 있는데?'},
 {'role': 'assistant',
  'content': '저는 다양한 방식으로 도움을 드릴 수 있어요! 예를 들면:\n\n1. **정보 제공**: 역사, 과학, 기술, 문화 등에 관한 정보를 제공해드릴 수 있습니다.\n2. **학습 지원**: 영어, 수학 등의 학습에 필요한 자료나 설명을 도와줄 수 있어요.\n3. **조언 및 팁**: 여행 계획, 건강 관리, 요리 레시피 등 실생활에 유용한 조언을 드릴 수 있습니다.\n4. **문제 해결**: 특정 문제나 상황에

In [ ]:
# 대화내역을 로깅
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')

    # messages에 assistant 응답 추가
    messages.append({'role': 'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕
Assistant: 안녕하세요! 만나서 반가워요😊 무엇을 도와드릴까요?
User: 흠
Assistant: 흠! 무언가 고민이 있으신가요, 아니면 그냥 심심하신가요? 궁금한 게 있으면 언제든 말씀해 주세요. 도움이 되고 싶어요! 😊
User: 하하
Assistant: 하하! 무슨 재미있는 일이 있으셨나요? 이야기해 주시면 같이 웃어드릴게요 😊
User: exit
채팅을 종료합니다...


###  stream

In [ ]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": "Hello!"}
    ],
    stream=True
)

full_reply = ""
for chunk in response:
    content_piece = chunk.choices[0].delta.content
    if content_piece:  # content가 있는 경우만
        print(content_piece, end="", flush=True)
        full_reply += content_piece

Hello! How can I assist you today?

## stream

In [ ]:
stream = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': '지금 stream테스트할거야, 아주 긴 응답 메세지를 보내줘.'}],
    stream=True
)
for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end='')

물론입니다! 아래는 꽤 긴 응답 메시지입니다:

---

안녕하세요! 당신에게 긴 메시지를 드리기 위해 준비했습니다. 지금부터 다양한 주제에 대해 이야기하겠습니다.

**1. 인류의 역사:**
인류의 역사는 수천 년에 걸쳐 펼쳐졌습니다. 선사 시대, 고대 문명, 중세, 근대, 현대 등으로 나누어 볼 수 있으며, 각 시대마다 인류는 놀라운 발전을 이루어 왔습니다. 약 200,000년 전, 초기 인간이 아프리카에서 출현한 이후, 인류는 농업, 산업, 정보 사회로 발전하게 됩니다. 이 과정에서 불, 도구, 언어, 문자를 개발하며 다양한 문화를 형성하였습니다.

**2. 과학과 기술:**
과학은 우리 삶에 큰 영향을 미치고 있습니다. 뉴턴의 고전 물리학, 아인슈타인의 상대성이론, 다윈의 진화론 등은 우리 세계관을 크게 변화시킨 이론들입니다. 현대 기술은 인터넷과 스마트폰의 발달로 사람들 간의 소통을 혁신적으로 변화시켰고, 인공지능(AI)과 머신러닝 기술의 발전은 다양한 산업에서 업무 방식에 큰 변화를 가져오고 있습니다.

**3. 환경과 지속 가능성:**
우리는 지금 환경 문제에 직면해 있습니다. 기후 변화, 오염, 생물 다양성 감소 등은 전 세계가 해결해야 할 중요한 주제입니다. 재생 가능한 에너지원의 사용 증가, 플라스틱 사용 감소, 친환경 농업 등의 노력은 이러한 문제를 해결하기 위한 지속 가능한 방법으로 주목받고 있습니다.

**4. 문화와 예술:**
문화는 인간의 삶의 질과 직결되는 중요한 요소입니다. 각 지역의 전통, 음악, 미술, 문학 등은 그 사회의 가치관과 정체성을 반영합니다. 예술은 감정을 표현하고 사회적 이슈를 조명하는 중요한 수단이기도 하며, 세계 여러 나라의 다양한 문화들이 서로 영향을 주고받으며 발전하고 있습니다.

**5. 사회와 경제:**
현대 사회는 다양한 사회적, 경제적 이슈들과 직면해 있습니다. 불평등, 실업, 고령화 사회 등은 모두가 공감할 수 있는 문제입니다. 이러한 문제를 해결하기 위해 정부와 기업, 비영리 단체 등이 다양한 접

In [ ]:
# stream 버젼
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    stream = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1,
        stream=True
    )
    print(f'Assistant: ', end='')
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='', flush=True) # 내부 buffer 사용하지 않음
    print()

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: ㄷㅌ
Assistant: 안녕하세요! "ㄷㅌ"라고 입력해주셨네요. 혹시 더 자세히 말씀해주실 수 있을까요? 궁금한 점이나 도움이 필요한 게 있다면 언제든 말씀해주세요! 😊
User: exit
채팅을 종료합니다...


# Token counting

비용 = ((입력 토큰수 * 단가) + (출력 토큰수 * 단가)) * 월 서비스 호출수

In [ ]:
!pip install tiktoken

In [ ]:
# 각 모델에 따른 토커나이져(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o')ㄷ
# gpt41 = tiktoken.encoding_for_model('gpt-4.1')
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35)
print(gpt4o)
print(gpt41)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>
<Encoding 'cl100k_base'>


In [ ]:
# 토큰수 세기
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다.')
encoded_gpt4o = gpt4o.encode('아버지가 방에 들어가신다.')

print(len(encoded_gpt35))
print(len(encoded_gpt4o))

13
10


## 토큰비용 계산하기

In [ ]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

In [ ]:
# 토큰수 세기
encoded_gpt35 = gpt35.encode(text)
encoded_gpt4o = gpt4o.encode(text)

print(len(encoded_gpt35))
print(len(encoded_gpt4o))

1215
703


In [90]:
response_gpt4o = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

output_gpt4o = response_gpt4o.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print("gpt-4o 응답: ", output_gpt4o)
print("gpt-4.1 응답: ", output_gpt41)

print('\n', 'gpt-4o 토큰수: ', len(gpt4o.encode(output_gpt4o)))
print('gpt-4.1 토큰수: ', len(gpt41.encode(output_gpt41)))

gpt-4o 응답:  더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성하는 입장을 시사하자, 탄핵소추안 표결 시점을 앞당기는 방안을 검토하고 있습니다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라며 신중한 태도를 보이고 있습니다. 민주당은 한 대표의 발언 이후 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 표결 시점을 당초 예정된 7일 오후 7시에서 5시로 앞당기는 방안도 논의 중입니다. 민주당은 국민의힘의 지연 전략 가능성을 염두에 두고 있으며, 상황에 따라 본회의 일정을 조정할 수 있다고 밝혔습니다.
gpt-4.1 응답:  ### 기사 핵심 요약

**1. 민주당, 윤석열 대통령 탄핵소추안 표결 시점 앞당길지 고심**
- 한동훈 국민의힘 대표가 윤 대통령 탄핵에 사실상 찬성하는 듯한 입장을 내비치자, 더불어민주당이 7일로 예정된 탄핵소추안 표결을 앞당기는 방안을 논의 중임.
- 이재명 민주당 대표는 한동훈 대표의 진짜 입장을 먼저 정확히 파악해야 한다며 신중한 태도를 보임.

**2. 긴급 의원총회 소집 및 비상 상황 인식**
- 한동훈 대표의 입장 변화 이후 민주당은 긴급 의원총회를 열어 당내 의견을 수렴하고 있음.
- 민주당은 최근 정치인 체포 시도 등 일련의 상황을 매우 비상하게 인식, 추가 돌발 상황(‘2차 계엄’ 등) 가능성도 우려.

**3. 표결 시점 변경 가능성**
- 공식적으로 본회의 일정 변경을 요청하진 않았으나, 필요시 의장실과 협의해 표결을 앞당길 수 있다고 밝힘.
- 실제로 7일 오후 7시로 예정된 표결을 오후 5시로 앞당기는 방안이 거론되고 있음. 이는 국민의힘의 지연 전략에 대응하기 위한 조치.

**4. 민주당의 신중한 대응**
- 민주당은 한동훈 대표의 입장과 국민의힘의 대응을 면밀히 파악한 뒤, 표결 시점 변경 등 구체적 행동에 나설 방침임.

---

**핵심 포인트:**  
한동훈 대표의 탄핵 관련 발언을 계기로 민주당이 윤 대통령 탄핵소추안 표결 시점을 앞당길지 고심 

In [99]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
    if model == 'gpt-4.1':
        encoding = tiktoken.get_encoding('cl100k_base')
    else:
        encoding = tiktoken.encoding_for_model(model)
    encoded = encoding.encode(text)
    return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)
    input_cost = (input_tokens * PRICING[model]['input']) / num_service_call
    output_cost = (output_tokens * PRICING[model]['output']) / num_service_call
    total_cost = input_cost + output_cost
    return input_cost, output_cost, total_cost

input_cost1, output_cost1, total_cost1 = calc_cost(text, output_gpt4o, model='gpt-4o-mini')

input_cost2, output_cost2, total_cost2 = calc_cost(text, output_gpt41, model='gpt-4.1')

print(f"gpt4o input cost: ${input_cost1:.6f}")
print(f"gpt4o output cost: ${output_cost1:.6f}")
print(f"gpt4o total cost: ${total_cost1:.6f}")


print(f"\ngpt4.1 input cost: ${input_cost2:.6f}")
print(f"gpt4.1 output cost: ${output_cost2:.6f}")
print(f"gpt4.1 total cost: ${total_cost2:.6f}")

gpt4o input cost: $0.000105
gpt4o output cost: $0.000107
gpt4o total cost: $0.000213

gpt4.1 input cost: $0.002430
gpt4.1 output cost: $0.005848
gpt4.1 total cost: $0.008278
